In [25]:
# Подключение и импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from PIL import Image
from torchvision import transforms
import glob

In [28]:
# Получаем датасет из репозитория
!git clone https://github.com/Aidt87/final_project_ai_architech_course.git

Cloning into 'final_project_ai_architech_course'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 170 (delta 1), reused 166 (delta 0), pack-reused 0
Receiving objects: 100% (170/170), 13.25 MiB | 20.08 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [29]:
# Настройки датасета(при запуске не через Colab, поменять путь к датасету)
data_dir = '/content/final_project_ai_architech_course/helmet_dataset'  # Каталог с изображениями
batch_size = 50  # Размер пакета данных для обучения

In [30]:
# Предобработка данных. Подготовка к обучению.
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Случайное изменение размера и обрезка до 224x224 пикселей
        transforms.RandomHorizontalFlip(),  # Случайное горизонтальное отражение изображения
        transforms.ToTensor(),  # Преобразование в тензор (многомерный массив)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  # Изменение размера до 256x256 пикселей
        transforms.CenterCrop(224),  # Обрезка до 224x224 пикселей по центру»
        transforms.ToTensor(),  # Преобразование в тензор
    ]),
}

# Создание ImageFolder датасета для обучения и валидации с использованием заданных трансформаций
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

# Создание DataLoader для загрузки данных с пакетами, перемешиванием и указанием числа рабочих процессов
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val']}

# Определение размеров датасетов для обучения и валидации
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# Получение списка классов (классификация: helmet, no_helmet)
class_names = image_datasets['train'].classes

# Загрузка предварительно обученной модели ResNet152
model = models.resnet152(pretrained=True)

# Получение количества признаков в последнем полносвязном слое
num_ftrs = model.fc.in_features

# Замена последнего полносвязного слоя на слой с 2 выходами (2 класса: helmet, no_helmet)
model.fc = nn.Linear(num_ftrs, 2)

# Определение устройства для обучения(GPU или CPU). При отсутствия к видеокарте проводим обучеие на процессоре.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Перемещение модели на выбранное устройство
model = model.to(device)

# Определение функции потерь (cross-entropy) и оптимизатора (SGD)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
# Обучение модели
# Количество эпох
num_epochs = 10

for epoch in range(num_epochs):
    # Тренируем модель
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        # стохастический градиентный спуск для обновления весов модели
        optimizer.zero_grad()
        outputs = model(inputs)
        # принимает предсказанные значения модели и истинные (ожидаемые) значения (метки) и вычисляет, насколько они различаются.
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / dataset_sizes['train']}")

# Сохраняем модель
torch.save(model.state_dict(), 'helmet_classification_model.pth')

Epoch 1, Loss: 0.018081421995985097
Epoch 2, Loss: 0.0174771249294281
Epoch 3, Loss: 0.015343422519749609
Epoch 4, Loss: 0.013322754666723054
Epoch 5, Loss: 0.011910173656611607
Epoch 6, Loss: 0.010409456131787136
Epoch 7, Loss: 0.01008207171127714
Epoch 8, Loss: 0.007989265538495162
Epoch 9, Loss: 0.007632298459266794
Epoch 10, Loss: 0.006768465555947402


In [32]:
# Классификация нового изображения

# Создаем функцию для классификации изображения
def classify_image(image_path):

    # Загружаем изображение
    image = Image.open(image_path)
    #
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    # Добавление размерности батча (batch dimension)
    image = preprocess(image).unsqueeze(0)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # Получаем наиболее вероятный резульат
    predicted_class = class_names[predicted[0]]
    if predicted_class == 'helmet':
      predicted_class = "Есть каска"
    else:
      predicted_class = "Нет каски"

    image_paths = image_path.split('/')
    image_name = image_paths[-1]

    print(f'Для изображения {image_name} прогнозируемый тип: {predicted_class}')

    return class_names[predicted[0]]

def classify_images(files, files_class = ''):
  all = len(files)
  correct = 0.0
  for file in files:
    # Используем функцию classify_image для классификации новых изображений
    predicted_class = classify_image(file)
    if files_class == predicted_class:
      correct += 1

  if files_class != '':
    print(f"Accuracy: {correct/all}")

In [33]:
# Проверяем на тестовых файлах
validations = glob.glob(os.path.join(data_dir, 'val', 'helmet', '*.*'))
classify_images(validations, 'helmet')
print()

validations = glob.glob(os.path.join(data_dir, 'val', 'no_helmet', '*.*'))
classify_images(validations, 'no_helmet')

Для изображения helmet4.jpg прогнозируемый тип: Есть каска
Для изображения helmet1.jpg прогнозируемый тип: Есть каска
Для изображения helmet3.jpg прогнозируемый тип: Есть каска
Для изображения helmet2.jpg прогнозируемый тип: Есть каска
Accuracy: 1.0

Для изображения no_helmet1.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet4.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet2.jpg прогнозируемый тип: Есть каска
Для изображения no_helmet5.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet3.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet6.jpg прогнозируемый тип: Нет каски
Accuracy: 0.8333333333333334


In [34]:
# Получаем список демонстрационных файлов
demonstrations = glob.glob(os.path.join(data_dir, 'demo', '*.*'))
classify_images(demonstrations)

Для изображения no_helmet9.jpg прогнозируемый тип: Нет каски
Для изображения helmet5.jpg прогнозируемый тип: Есть каска
Для изображения helmet4.jpg прогнозируемый тип: Есть каска
Для изображения no_helmet12.jpg прогнозируемый тип: Нет каски
Для изображения helmet8.jpg прогнозируемый тип: Есть каска
Для изображения helmet1.jpg прогнозируемый тип: Есть каска
Для изображения helmet3.jpg прогнозируемый тип: Есть каска
Для изображения helmet9.jpg прогнозируемый тип: Есть каска
Для изображения no_helmet8.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet1.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet7.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet4.jpg прогнозируемый тип: Нет каски
Для изображения helmet7.jpg прогнозируемый тип: Есть каска
Для изображения helmet11.jpg прогнозируемый тип: Есть каска
Для изображения no_helmet2.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet5.jpg прогнозируемый тип: Нет каски
Для изображения no_helmet13.jpg прогно